In [164]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

In [165]:

file=open(data_directory+data_file,"r+")
str_input=file.read()


In [166]:
unq_chars={j:i for (i,j) in enumerate(sorted([i for i in set(str_input)]))}
index_to_chars={j:i for (i,j) in unq_chars.items()}

vocab_size=len(index_to_chars)

In [167]:
T=str_input
length=len(T) #1,29992
batch_chars=length/batch_size #(16)
batch_chars=int(round(batch_chars,0))

In [178]:
#Directory for data 
data_directory = "Data/"
#includes input.txt which includes abc-notation music (ABC version of the nottingham music database (jigs (340 tunes)))
data_file = "input.txt"
#index file to storedistinct vocab 
charIndex_json = "char_to_index.json"
#Saving model weights
model_weights_directory = 'Data/Model_Weights/'
#batch size
batch_size = 16
#sequence length
seq_length = 64
MODEL_DIR='./model'


In [169]:
def save_weights(epoch,model):
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR)
    model.save_weights(os.path.join(MODEL_DIR,'weights.{}.h5'.format(epoch)))

def load_weights(epoch,model):
    model.load_weights(os.path.join(MODEL_DIR,'weights.{}.h5'.format(epoch)))



In [170]:
def read_batches(inp,vocab_size):
    for i in range(0,batch_chars-seq_length,seq_length):
        X=np.zeros((batch_size,seq_length))
        Y=np.zeros((batch_size,seq_length,vocab_size))
        for j in range(0,batch_size):
            for k in range(0,seq_length):
                X[j,k]=inp[batch_chars*j+i+k]
                Y[j,k,inp[batch_chars*j+i+k+1]]=1
        yield X,Y

In [171]:
def build_model(batch_size, seq_length, unq_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unq_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    
    for i in range(3):
        model.add(LSTM(256, return_sequences = True, stateful = True))
        model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))

    model.add(Activation("softmax"))
    
    return model

if __name__=='__main__':
    model=build_model(16,64,50)
    model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (16, 64, 512)             25600     
_________________________________________________________________
lstm_4 (LSTM)                (16, 64, 256)             787456    
_________________________________________________________________
dropout_4 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_5 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_5 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_6 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_6 (Dropout)          (16, 64, 256)            

In [257]:
def train(text, epochs = 1,save_fre=10):

    
    #model architecture
    model = built_model(batch_size,seq_length, vocab_size)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    losses=[]
    accs=[]
    epoch_number=[]
    print(losses)
    #Train data generation
    inp=np.asarray([unq_chars[c] for  c in text],dtype=np.int32)
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))

        for i, (X, Y) in enumerate(read_batches(inp, vocab_size)):

            loss, acc = model.train_on_batch(X, Y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            losses.append(loss)
            accs.append(acc)
            epoch_number.append(epoch+1)
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, loss, acc))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
           
       
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
   #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame.loc[epoch,"Epoch"] = epoch+1
    log_frame.loc[epoch,"Loss"] = np.average(losses)
    log_frame.loc[epoch,"Accuracy"] = np.average(accs)
    log_frame.to_csv("log.csv", index = False)
  

In [258]:
k=train(str_input)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (16, 64, 512)             44544     
_________________________________________________________________
lstm_55 (LSTM)               (16, 64, 256)             787456    
_________________________________________________________________
dropout_55 (Dropout)         (16, 64, 256)             0         
_________________________________________________________________
lstm_56 (LSTM)               (16, 64, 256)             525312    
_________________________________________________________________
dropout_56 (Dropout)         (16, 64, 256)             0         
_________________________________________________________________
lstm_57 (LSTM)               (16, 64, 256)             525312    
_________________________________________________________________
dropout_57 (Dropout)         (16, 64, 256)           

Batch: 115, Loss: 2.791325569152832, Accuracy: 0.2470703125
Batch: 116, Loss: 2.8140859603881836, Accuracy: 0.2353515625
Batch: 117, Loss: 2.8305163383483887, Accuracy: 0.2373046875
Batch: 118, Loss: 2.7980198860168457, Accuracy: 0.23046875
Batch: 119, Loss: 2.8383116722106934, Accuracy: 0.240234375
Batch: 120, Loss: 2.7792344093322754, Accuracy: 0.2490234375
Batch: 121, Loss: 2.6455259323120117, Accuracy: 0.283203125
Batch: 122, Loss: 2.72941255569458, Accuracy: 0.2646484375
Batch: 123, Loss: 2.7642486095428467, Accuracy: 0.2451171875
Batch: 124, Loss: 2.686966896057129, Accuracy: 0.259765625
Batch: 125, Loss: 2.655385971069336, Accuracy: 0.27734375
Batch: 126, Loss: 2.668875217437744, Accuracy: 0.2861328125


In [228]:

from keras.utils import plot_model
plot_model(model, to_file='model.png',show_shapes=True, show_layer_names=True)

NameError: name 'losses' is not defined